# Breve tour do Fatiando

## Formas geométricas

## Modelagem direta

## Dados de gravidade

In [ ]:
lon, lat, y, x, alt, topo, grav = np.loadtxt('gravidade-havai.txt', unpack=True)

## Dados magnéticos

In [ ]:
x, y, z, anom = np.loadtxt('anomalia-mag.txt', unpack=True)
shape = (50, 50)

## TGA (sinal analítico)

## Derivadas

## Deconvolução de Euler

## Redução ao polo

## Direção de magnetização